## Importing modules and reading csv file

In [70]:
import os
import polars as pl
from utils import *
import pandas as pd
from datetime import datetime

Let's read the data corresponding to a single state ('AC' in this case) to explore it.

In [ ]:
csv_files_dir = '../data/csv_files'
file = 'RS.csv'
file_path = os.path.join(csv_files_dir, file)
df = read_csv_pl(file_path, separator = ',')
pdf = df.to_pandas()

Let's check the different types of 'ClassInfraFisica' present.

In [ ]:
df.group_by('ClassInfraFisica').count()

Let's fix the typo:

In [ ]:
df = strip_column_pl(df, 'ClassInfraFisica')

In [ ]:
df.group_by('ClassInfraFisica').agg(pl.col('ClassInfraFisica').count().alias('Count'))

In [ ]:
df = replace_values_pl(df, 'ClassInfraFisica', 'Greenfild', 'Greenfield')

## Investigating the relationship between ClassInfraFisica and NumEstacao

Now, we have to check whether each station (indicated by the column 'NumEstacao') corresponds to a single type of 'ClassInfraFisica'; which is the expected behavior (assumption).

In [ ]:
count_unique_values = df.group_by('NumEstacao').agg(pl.col('ClassInfraFisica').n_unique().alias('unique_count'))
instances_with_different_values = count_unique_values.filter(count_unique_values['unique_count'] > 1)
print(instances_with_different_values)

We found that we can actually get two different 'ClassInfraFisica' for any individual station. But if we check further, we'll see that it's actually pairing types with 'null' values, which does not compromise the assumption.

In [ ]:
df.filter(df['NumEstacao'] == 1007447238)[['NumEstacao', 'ClassInfraFisica']]

Let's group by 'NumEstacao', but keeping all the info present in the rows as sets (or as a single value if all rows match)

In [ ]:
def set_aggregation(x):

    if len(set(x)) == 1:
        return x.iloc[0]  
    else:
        return set(x)  

dfg = pdf.groupby('NumEstacao').agg(lambda x: set_aggregation(x))
dfg.head(4)

In [ ]:
print(f"Percentage of initial rows kept when grouping = {len(dfg)*100/len(df):.2f}%")

Now we have a dataframe with all the original information, but only 16% of the rows; in which each represents a single station

## Understanding the Columns

Let's understand what the columns are about. This will futurely enable us to better make sense of them, process them if necessary, or drop irrelevant ones.

From [Página de Emissões - Anatel](https://sistemas.anatel.gov.br/anexar-api/publico/anexos/download/9cfc11fc83fcfc2d2586cdb887f72cb5), we can understand what each column refers to.

Status.state: Situação da solicitação na Anatel.

NumFistel: É um código numérico definido pela Anatel, composto de onze dígitos que identifica a autorização que a empresa possui para determinado serviço em determinada região.

NomeEntidade: Nome da Empresa detentora da estação.

NumServico: Código do serviço de telecomunicações na Anatel.

NumAto: Número do Ato de Autorização de Uso de Radiofrequência, sendo que os quatro últimos dígitos se referem ao ano da assinatura do respectivo ato. Exemplo XYZW2017, ato XYZW de 2017.

EnderecoEstacao: Endereço completo onde a estação está instalada.

EndComplemento: Complemento do endereço, caso haja.

DesignacaoEmissao: Baseado em [Método de Designação - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60403&assuntoPublicacao=null&caminhoRel=null&filtro=1&documentoPath=outros/autocadastramento/metodo_de_designacao.pdf)

Designação de emissão. Para uma completa designação da emissão, necessitamos sempre de 9 caracteres alfanuméricos. Os quatros primeiros representam a _largura de faixa necessária_; os três seguintes as _características básicas_ e os dois últimos as _características adicionais facultativas_.
- Largura de faixa necessária: Para uma dada classe de emissão, o valor mínimo da largura de faixa ocupada pela emissão, suficiente para garantir a transmissão da informação com a velocidade de transmissão e com a qualidade requerida para o sistema empregado, nas condições especificadas. A largura de faixa necessária será sempre expressa por meio de três algarismos, que indicam os três primeiros algarismos significativos de largura de faixa necessária e uma letra que ocupa a posição da virgula decimal e representa a Unidade de largura de faixa e será H para Hertz, K para Kilohertz, M para Megahertz ou G para Gigahertz.
- Características básicas: descritas por três símbolos:
    - Primeiro símbolo: tipo de modulação da portaria principal.
    - Segundo símbolo: natureza do(s) sinais que modulam a portaria principal.
    - Terceiro símbolo: tipo de informação a ser transmitida.
- Características adicionais facultativas: Para descrição mais completa de uma emissão, são previstas duas características facultativas, as quais são expressas pelos quarto e quinto símbolos. Quando não se utiliza o quarto ou o quinto símbolo, convém indicar isso mediante um traço no lugar em que cada símbolo apareceria.

meioAcesso: é um campo de ‘estações dispensadas de licenciamento’. As opções são: fibra, par metálico, cabo coaxial e radiação restrita.

Azimute: posicionamento m graus em relação ao Norte do lóbulo principal de radiação da antena. Quando for utilizada antena omnidirecional será 0.

CodTipoClasseEstacao: Classe da Estação conforme lista permitida no sistema de canalização do Mosaico, conforme o Regulamento da faixa de frequência utilizada conforme Manual de Projetos Técnicos do SITAR, disponível em [Manual de Projeto - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60402&assuntoPublicacao=MANUAL%20DE%20PROJETOS%20T%C9CNICOS%20(SITAR)&caminhoRel=CidadaoComunica%E7%E3o%20via%20R%E1dioServi%E7o%20Limitado&filtro=1&documentoPath=outros/). Exemplos:
- ML: Estação móvel terrestre
- FX: Estação fixa
- FB: Estação de base
- XR: Estação fixa repetidora
- FA: Estação aeronáutica
- RC: Radiofarol não direcional

ClassInfraFisica: Classificação de Infraestrutura Física. Apenas para estações dos serviços de interesse coletivo (STFC, SCM e SMP). Para serviços de interesse restrito (SLP), estará em branco. Valores válidos: Greenfield, Streetlevel, Ran Sharing, Small Cell, COW, Picocelula, Harmonizada, Rooftop, Indoor, Outdoor

CompartilhamentoInfraFisica: Compartilhamento de Infraestrutura Física. Apenas para estações dos serviços de interesse coletivo (não se aplica para SLP).

CodTipoAntena: Tipo Antena: campo obrigatório, onde é inserido o código conforme item 2.3.4 do [Manual de Projeto - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60402&assuntoPublicacao=MANUAL%20DE%20PROJETOS%20T%C9CNICOS%20(SITAR)&caminhoRel=CidadaoComunica%E7%E3o%20via%20R%E1dioServi%E7o%20Limitado&filtro=1&documentoPath=outros/). Exemplos:
- 019: Monopolo vertical
- 060: V invertido
- 213: Cabo fendido

GanhoAntena: Ganho da Antena em dB. Se a frequência for superior a 28.000 kHz, é informado o ganho da antena em relação a uma antena isotrópica (dbi); se inferior, este é dado em relação a uma antena dipolo (dBd).

FrenteCostaAntena: Relação Frente/Costa em dBi (campo obrigatório). Deve ser menor que 90 dbi. Será 0 (zero) quando for utilizada antena omnidirecional.

AnguloMeiaPotenciaAntena: Ângulo de meia potência em graus decimais (campo obrigatório). Pontos no diagrama onde a potência radiada equivale à metade da radiada na direção principal.

AnguloElevacao: : Ângulo de elevação mecânico em graus decimais. O valor deve estar entre 0 e 90 e ser medido com relação à linha de horizonte, sendo negativo quando a linha de visada estiver abaixo desta referência.

Polarizacao:  a orientação do campo elétrico da onda de rádio com respeito a terra ou direção de propagação; e é determinado pela estrutura física da antena e por sua orientação. Valores aceitos: H (horizontal), V (vertical), CR (circular á direita), CL (circular à esquerda) e X (horizontal e vertical simultâneas ou não).

AlturaAntena: Altura da antena (em relação a cota do terreno). Deve ser menor que 200 m.

PotenciaTransmissorWatts: Potência nominal (saída do transmissor) em watts.

CodDebitoTFI: Código da Taxa de Fiscalização de Instalação (TFI) conforme lei nº 5.070, de 7 de julho de 1966.
Taxa de fiscalização da instalação é aquela devida pelas concessionárias e permissionárias de serviços de telecomunicações, no momento em que lhes é outorgada autorização para a execução do serviço e tem a finalidade de ressarcir as despesas realizadas pelo Poder Público até o licenciamento das respectivas estações. Não serão licenciadas as estações das permissionárias e concessionárias de serviços de telecomunicações que não efetuarem o pagamento da taxa de fiscalização da instalação.
[Lei do Fistel](https://www2.camara.leg.br/legin/fed/lei/1960-1969/lei-5070-7-julho-1966-364619-anexo-pl.pdf)

DataLicenciamento:  Data do último licenciamento da estação. Formato: AAAA/MM/DD

DataPrimeiroLicenciamento: Data do primeiro licenciamento da estação. Formato: AAAA/MM/DD

NumRede: Valor numérico, definido pelo usuário, para identificar grupo de estações que se comunicam. Visa facilitar a análise da comunicação das redes.

_id:  Identificação da emissão / frequência da estação

DataValidade: Validade da RF associada àquela estação.

NumFistelAssociado: Campo preenchido quando a emissão for de outro Fistel (do mesmo CNPJ ou diferentes): licenciamento conjunto. Nesse caso, é informado o número do Fistel responsável pelo Ato de RF da emissão dessa linha



meioAcesso consists of all None values

In [ ]:
pdf['meioAcesso'].value_counts(dropna=False)

In [ ]:
# CodEquipamentoTransmissor
print(f"number of rows: {len(pdf['CodEquipamentoTransmissor'])}")
print(f"number of unique rows: {pdf['CodEquipamentoTransmissor'].nunique()}")

In [ ]:
# CodDebitoTFI
pdf['CodDebitoTFI'].value_counts()

In [ ]:
# _id
pdf['_id'].value_counts()

In [ ]:
# DataValidade
pdf['DataValidade'].value_counts()

In [ ]:
pdf['NumFistelAssociado'].value_counts()

In [ ]:
pdf['NomeEntidadeAssociado'].value_counts()

## Filtering out irrelevant columns

In [ ]:
pdf = filter_columns(pdf)

pdf.head()

### NomeEntidade

Given the high number of different providers found in this column (sometimes it's the mobile network operator, sometimes it's the venue owner), we will not be using this column.

## Column Treatment

Now we will correctly treat each column in order to have relevant information and prepare it for training.

In [73]:
import os
import polars as pl
from utils import *
import pandas as pd
csv_files_dir = '../data/csv_files'
file = 'MS.csv'
file_path = os.path.join(csv_files_dir, file)
df = read_csv_pl(file_path, separator = ',')
pdf = df.to_pandas()

### SiglaUf

In [ ]:
pdf['SiglaUf']

### CodMunicipio

In [ ]:
pdf['CodMunicipio'] = pdf['CodMunicipio'].fillna(0).astype(int)
pdf['CodMunicipio']

### DesignacaoEmissao

In [2]:
pdf[['LarguraFaixaNecessaria', 'CaracteristicasBasicas']] = pdf.apply(process_designacao_emissao, axis=1)
pdf.drop(columns=['DesignacaoEmissao'], inplace=True)

### Tecnologia & tipoTecnologia

In [8]:
pdf['Tecnologia_e_Tipo'] = pdf.apply(lambda row: concatenate_columns(row, 'Tecnologia', 'tipoTecnologia'), axis=1)
pdf.drop(columns=['Tecnologia', 'tipoTecnologia'], inplace=True)

In [7]:
x[~x['tipoTecnologia'].isnull()]

,Tecnologia,tipoTecnologia,Tecnologia2
316,NR,NSA,NR_NSA
319,NR,NSA,NR_NSA
322,NR,NSA,NR_NSA
3136,NR,SA-NSA,NR_SA-NSA
3137,NR,SA-NSA,NR_SA-NSA
...,...,...,...
56664,NR,NSA,NR_NSA
56665,NR,NSA,NR_NSA
56666,NR,NSA,NR_NSA
56667,NR,NSA,NR_NSA


### FreqTxMHz

In [9]:
pdf['FreqTxMHz']

0         874.5
1         874.5
2         874.5
3        2680.0
4        2680.0
          ...  
56673    1867.5
56674    1867.5
56675    1867.5
56676    1867.5
56677    1867.5
Name: FreqTxMHz, Length: 56678, dtype: float64

### FreqRxMHz

In [11]:
pdf['FreqRxMHz']

0         829.5
1         829.5
2         829.5
3        2560.0
4        2560.0
          ...  
56673    1772.5
56674    1772.5
56675    1772.5
56676    1772.5
56677    1772.5
Name: FreqRxMHz, Length: 56678, dtype: float64

### CodTipoClasseEstacao

In [13]:
pdf['CodTipoClasseEstacao'].value_counts()

CodTipoClasseEstacao
ML    36685
FB    13457
FX     6220
BR      179
XR       68
TX       45
AM       12
FA        8
RC        2
FR        2
Name: count, dtype: int64

### ClassInfraFisica

In [26]:
pdf['ClassInfraFisica'].value_counts(dropna=False)

ClassInfraFisica
None            271007
Greenfield       45178
Rooftop           5307
Ran Sharing        750
Indoor             400
COW                129
SmallCell          114
Harmonizada         87
Streetlevel         71
Outdoor              9
Streetlevel          9
Name: count, dtype: int64

In [27]:
pdf = strip_column(pdf, 'ClassInfraFisica')
pdf['ClassInfraFisica'] = pdf['ClassInfraFisica'].str.upper()
pdf = replace_values(pdf, 'ClassInfraFisica', 'GREENFILD', 'GREENFIELD')
pdf['ClassInfraFisica'].value_counts(dropna=False)

ClassInfraFisica
None           271007
GREENFIELD      45178
ROOFTOP          5307
RAN SHARING       750
INDOOR            400
COW               129
SMALLCELL         114
HARMONIZADA        87
STREETLEVEL        80
OUTDOOR             9
Name: count, dtype: int64

### CompartilhamentoInfraFisica

In [29]:
pdf['CompartilhamentoInfraFisica'].value_counts(dropna=False)

CompartilhamentoInfraFisica
None    286710
NA       22900
sim      13386
não         65
Name: count, dtype: int64

### CodTipoAntena

In [31]:
pdf['CodTipoAntena'].value_counts()

CodTipoAntena
19.0     138394
760.0     88494
655.0     21958
167.0     15605
817.0     12944
78.0      12168
728.0      5858
21.0       4670
752.0      3926
35.0       3155
396.0      2462
558.0      2212
574.0      1815
809.0      1772
86.0       1698
612.0      1041
469.0       948
36.0        617
37.0        552
604.0       398
586.0       394
736.0       362
566.0       268
38.0        257
168.0       226
20.0        168
590.0       117
582.0        95
400.0        94
744.0        65
94.0         56
795.0        54
540.0        54
175.0        37
397.0        32
620.0        19
353.0        14
840.0        12
191.0        10
213.0         9
787.0         8
779.0         6
642.0         6
450.0         2
833.0         2
27.0          1
656.0         1
657.0         1
658.0         1
796.0         1
Name: count, dtype: int64

In [33]:
pdf['CodTipoAntena'] = pdf['CodTipoAntena'].fillna(0).astype(int)
pdf['CodTipoAntena'] = pdf['CodTipoAntena'].astype(str)
pdf['CodTipoAntena'].value_counts()

CodTipoAntena
19     138394
760     88494
655     21958
167     15605
817     12944
78      12168
728      5858
21       4670
752      3926
35       3155
396      2462
558      2212
574      1815
809      1772
86       1698
612      1041
469       948
36        617
37        552
604       398
586       394
736       362
566       268
38        257
168       226
20        168
590       117
582        95
400        94
744        65
94         56
795        54
540        54
175        37
397        32
620        19
353        14
840        12
191        10
213         9
787         8
642         6
779         6
0           2
450         2
833         2
656         1
657         1
658         1
27          1
796         1
Name: count, dtype: int64

### GanhoAntena

In [40]:
pdf['GanhoAntena'].value_counts()

GanhoAntena
2.1      153648
0         33679
14         4862
12         4365
17.5       3705
          ...  
-14           1
40.0          1
31.9          1
13.47         1
41,3          1
Name: count, Length: 440, dtype: int64

In [42]:
pdf['GanhoAntena'] = pd.to_numeric(pdf['GanhoAntena'], errors='coerce')
pdf['GanhoAntena']

0         16.85
1         16.85
2         16.85
3         13.42
4         13.42
          ...  
282693    16.15
282694    16.15
282695    16.15
282696    16.15
282697    16.15
Name: GanhoAntena, Length: 282698, dtype: float64

### FrenteCostaAntena

In [48]:
pdf['FrenteCostaAntena'] = pd.to_numeric(pdf['FrenteCostaAntena'], errors='coerce')
pdf['FrenteCostaAntena']

0         28.0
1         28.0
2         28.0
3         28.0
4         28.0
          ... 
282693    25.0
282694    25.0
282695    25.0
282696    25.0
282697    25.0
Name: FrenteCostaAntena, Length: 282698, dtype: float64

### AnguloMeiaPotenciaAntena

In [50]:
pdf['AnguloMeiaPotenciaAntena'] = pd.to_numeric(pdf['AnguloMeiaPotenciaAntena'], errors='coerce')
pdf['AnguloMeiaPotenciaAntena']

0         57.15
1         57.15
2         57.15
3         65.20
4         65.20
          ...  
282693    65.00
282694    65.00
282695    65.00
282696    65.00
282697    65.00
Name: AnguloMeiaPotenciaAntena, Length: 282698, dtype: float64

### AnguloElevacao

In [52]:
pdf['AnguloElevacao'] = pd.to_numeric(pdf['AnguloElevacao'], errors='coerce')
pdf['AnguloElevacao']

0        -1.0
1        -1.0
2        -3.0
3        -7.0
4        -7.0
         ... 
282693   -4.0
282694   -4.0
282695   -4.0
282696   -4.0
282697   -4.0
Name: AnguloElevacao, Length: 282698, dtype: float64

# Polarizacao

In [57]:
pdf['Polarizacao'] = pdf['Polarizacao'].str.upper()
pdf['Polarizacao']

0         X
1         X
2         X
3         X
4         X
         ..
282693    X
282694    X
282695    X
282696    X
282697    X
Name: Polarizacao, Length: 282698, dtype: object

### AlturaAntena

In [61]:
pdf['AlturaAntena'] = pd.to_numeric(pdf['AlturaAntena'], errors='coerce')
pdf['AlturaAntena']

0         48.0
1         48.0
2         48.0
3         32.0
4         32.0
          ... 
282693    40.0
282694    40.0
282695    40.0
282696    40.0
282697    40.0
Name: AlturaAntena, Length: 282698, dtype: float64

### CodEquipamentoTransmissor

In [63]:
pdf['CodEquipamentoTransmissor'].value_counts()

CodEquipamentoTransmissor
1.313131e+10    111824
1.522131e+10     41840
6.291401e+09      9984
8.307180e+10      7115
4.881190e+10      4863
                 ...  
1.020202e+09         1
8.470402e+09         1
1.232120e+10         1
2.429110e+10         1
1.530100e+10         1
Name: count, Length: 521, dtype: int64

### PotenciaTransmissorWatts

In [66]:
pdf['PotenciaTransmissorWatts'].value_counts(dropna=False)

PotenciaTransmissorWatts
3.000      102195
10.000      62507
40.000      31325
5.000       13427
1.800       11056
            ...  
200.720         1
200.730         1
200.740         1
200.750         1
200.104         1
Name: count, Length: 315, dtype: int64

### CodDebitoTFI

In [68]:
pdf['CodDebitoTFI'].value_counts(dropna=False)

CodDebitoTFI
D    196376
G     52309
C     15036
E      8767
A      7996
B      1916
F       298
Name: count, dtype: int64

### DataLicenciamento

In [74]:
pdf['DataLicenciamento'] = pd.to_datetime(pdf['DataLicenciamento'], format='%Y-%m-%d')

# Calculate the age in days
pdf['IdadeLicenciamento'] = (datetime.now() - pdf['DataLicenciamento']).dt.days

# Display the transformed DataFrame
print(pdf[['DataLicenciamento', 'IdadeLicenciamento']])

ValueError: unconverted data remains when parsing with format "%Y-%m-%d": " 11:38:26.833", at position 212. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.